In [207]:
import pandas as pd
from pathlib import Path
import yfinance as yf
import numpy as np
import csv

import warnings
warnings.filterwarnings('ignore')

In [165]:
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df_500 = payload[0]

In [185]:
tickers_500 = list(df_500["Symbol"])

In [234]:
season_start_month = ["2","5","8","11"]

In [279]:
start_date = "2019-12"
end_date = "2020-06"

In [280]:
%%time
momentum_list = []
id_list = []
_fail_symbols=list()
l = 0

for ticker in tickers_500:
    try:
        p = 0
        n = 0
        # Return the historical data
        price = yf.Ticker(ticker).history(period="2y")
        
        price_2020 = price.loc[start_date:end_date]
        
        # Generate the simple return for each day
        price_2020["return"] = price_2020["Close"].pct_change()

        # Calculate the number of days for both having positive and negative return
        for ret in price_2020["return"]:
            if ret >= 0:
                p = p+1
            else:
                n = n+1
        
        # Calculate the sign of cumulative return of the ticker
        cum_ret = np.sign((1 + price_2020["return"]).cumprod().tail(1)[0])

        # %Negative day - %Positive day
        n_p = n/(n+p) - p/(n+p)
        
        # Quality Momentum measurement
        QM = cum_ret * n_p
        id_list.append(QM)
        
        # Calculate the momentum indicator for the ticker
        momentum = np.prod(1 + price_2020["Close"].groupby(by=[price_2020.index.month]).tail(1).pct_change()) - 1
        momentum_list.append(momentum)
        
        l = l+1
        print(l, ticker)
    except KeyError as e:
        print(ticker, e)
        _fail_symbols.append(ticker)
        

1 MMM
2 ABT
3 ABBV
4 ABMD
5 ACN
6 ATVI
7 ADBE
8 AMD
9 AAP
10 AES
11 AFL
12 A
13 APD
14 AKAM
15 ALK
16 ALB
17 ARE
18 ALGN
19 ALLE
20 LNT
21 ALL
22 GOOGL
23 GOOG
24 MO
25 AMZN
26 AMCR
27 AEE
28 AAL
29 AEP
30 AXP
31 AIG
32 AMT
33 AWK
34 AMP
35 ABC
36 AME
37 AMGN
38 APH
39 ADI
40 ANSS
41 ANTM
42 AON
43 AOS
44 APA
45 AAPL
46 AMAT
47 APTV
48 ADM
49 ANET
50 AJG
51 AIZ
52 T
53 ATO
54 ADSK
55 ADP
56 AZO
57 AVB
58 AVY
59 BKR
60 BLL
61 BAC
62 BBWI
63 BAX
64 BDX
65 BBY
66 BIO
67 TECH
68 BIIB
69 BLK
70 BK
71 BA
72 BKNG
73 BWA
74 BXP
75 BSX
76 BMY
77 AVGO
78 BR
79 CHRW
80 COG
81 CDNS
82 CZR
83 CPB
84 COF
85 CAH
86 KMX
87 CCL
88 CARR
89 CTLT
90 CAT
91 CBOE
92 CBRE
93 CDW
94 CE
95 CNC
96 CNP
97 CERN
98 CF
99 CRL
100 SCHW
101 CHTR
102 CVX
103 CMG
104 CB
105 CHD
106 CI
107 CINF
108 CTAS
109 CSCO
110 C
111 CFG
112 CTXS
113 CLX
114 CME
115 CMS
116 KO
117 CTSH
118 CL
119 CMCSA
120 CMA
121 CAG
122 COP
123 ED
124 STZ
125 COO
126 CPRT
127 GLW
128 CTVA
129 COST
130 CCI
131 CSX
132 CMI
133 CVS
134 DHI
135 DHR
1

In [281]:
len(id_list)

502

In [282]:
for ticker in _fail_symbols:
    tickers_500.remove(ticker)
    print(ticker)

In [283]:
d = {"Tickers":tickers_500,"Momentum":momentum_list, "ID":id_list}
df = pd.DataFrame(data=d)

In [284]:
df

,Tickers,Momentum,ID
0,MMM,-0.098947,0.041096
1,ABT,0.061536,-0.068493
2,ABBV,0.140620,-0.109589
3,ABMD,0.416027,-0.041096
4,ACN,0.028270,-0.013699
...,...,...,...
497,YUM,-0.128794,-0.041096
498,ZBRA,0.001997,0.013699
499,ZBH,-0.198917,0.041096
500,ZION,-0.332707,0.027397


In [285]:
momentum_100 = df.sort_values(by="Momentum", ascending=False).head(100)

In [286]:
QM_50 = momentum_100.sort_values(by="ID").head(50)

In [287]:
momentum_100.head(10)

,Tickers,Momentum,ID
315,MRNA,2.282720,-0.013699
437,TSLA,1.581240,-0.136986
175,ETSY,1.397969,-0.109589
141,DXCM,0.853342,-0.123288
167,ENPH,0.820513,-0.123288
395,REGN,0.660941,-0.095890
345,NVDA,0.616280,-0.150685
360,PYPL,0.610705,-0.082192
485,WST,0.514100,-0.109589
24,AMZN,0.492997,-0.164384


In [288]:
QM_50.head(10)

,Tickers,Momentum,ID
359,PAYC,0.169852,-0.205479
112,CLX,0.446246,-0.205479
80,CDNS,0.383506,-0.205479
426,SNPS,0.400862,-0.205479
53,ADSK,0.303772,-0.191781
428,TMUS,0.328105,-0.191781
243,ILMN,0.116386,-0.191781
320,MNST,0.090795,-0.178082
313,MSFT,0.297617,-0.178082
6,ADBE,0.319881,-0.178082


In [289]:
momentum_100.to_csv("./Data/momentum_100_2019-12-2020-06.csv")
QM_50.to_csv("./Data/QM_50_2019-12-2020-06.csv")

In [229]:
spy = yf.Ticker("spy").history(period="2y")["Close"].loc["2021-02":"2021-04"].pct_change().cumsum().apply(np.exp)
spy

Date
2021-02-01         NaN
2021-02-02    1.014241
2021-02-03    1.015038
2021-02-04    1.026641
2021-02-05    1.030690
                ...   
2021-04-26    1.116164
2021-04-27    1.115923
2021-04-28    1.115603
2021-04-29    1.122735
2021-04-30    1.115382
Name: Close, Length: 63, dtype: float64